In [3]:
from pulp import *

In [ ]:
# stanoveni primalu

suppliers  = ['S1', 'S2']
depots     = ['D1', 'D2', 'D3']
customers  = ['C1', 'C2', 'C3', 'C4']

capacity = {'S1': 35, 'S2': 45}
demand   = {'C1': 20, 'C2': 25, 'C3': 15, 'C4': 20}

# koeficienty promennych
cost_SD = {              # první fáze
    ('S1','D1'):4, ('S1','D2'):6, ('S1','D3'):5,
    ('S2','D1'):3, ('S2','D2'):2, ('S2','D3'):4
}

# koeficiekty promennych
cost_DC = {              # druhá fáze
    ('D1','C1'):3, ('D1','C2'):4, ('D1','C3'):2, ('D1','C4'):3,
    ('D2','C1'):5, ('D2','C2'):2, ('D2','C3'):4, ('D2','C4'):1,
    ('D3','C1'):4, ('D3','C2'):3, ('D3','C3'):3, ('D3','C4'):2
}

# S->D
# D1  D2  D3 kapacita
# S1  4   6   5  35
# S2  3   2   4  45

# D->C
#     C1  C2  C3  C4
# D1  3   4   2   3
# D2  5   2   4   1
# D3  4   3   3   2
#     20  25  15  20
#        odbery

min **c**<sup>T</sup>x  
s.t. A**x**<=**b**  
**x**>=0

In [5]:
# příprava modelu
model = LpProblem("TwoStageTransport", LpMinimize)
# S -> D
x = LpVariable.dicts("x", (suppliers, depots), lowBound=0)
# D -> C
y = LpVariable.dicts("y", (depots, customers), lowBound=0)

In [6]:
# objective function: cílová funkce (minimalizace nákladů)

model += (
    lpSum(cost_SD[i,k] * x[i][k] for i in suppliers for k in depots) +
    lpSum(cost_DC[k,j] * y[k][j] for k in depots    for j in customers)
), "TotalCost"


minimalizuj =  
4 x<sub>11</sub> + 6 x<sub>12</sub> + 5 x<sub>13</sub> **S->D**  
+3 x<sub>21</sub> + 2<sub>22</sub> + 4 x<sub>23</sub> **S->D**  
+3 y<sub>11</sub> + 4 y<sub>12</sub> + 2 y<sub>13</sub> + 3 y<sub>14</sub> **D->C**  
+5 y<sub>21</sub> + 2 y<sub>22</sub> + 4 y<sub>23</sub> + 1 y<sub>24</sub> **D->C**  
+4 y<sub>31</sub> + 3 y<sub>32</sub> + 3 y<sub>33</sub> + 2 y<sub>34</sub> **D->C**  
  

c, d = koeficient  
x, y = množství  
∑<sub>ij</sub> c<sub>ij</sub>x<sub>ij</sub> + ∑<sub>jk</sub> d<sub>ij</sub>y<sub>ij</sub>

In [7]:
# constraints: omezení proměnných

# kapacita dodavatelů
for i in suppliers:
    model += lpSum(x[i][k] for k in depots) <= capacity[i], f"supply_{i}"
# rovnováha v depech
for k in depots:
    model += (
        lpSum(x[i][k] for i in suppliers) ==
        lpSum(y[k][j] for j in customers)
    ), f"balance_{k}"
# poptávka zákazníků
for j in customers:
    model += lpSum(y[k][j] for k in depots) == demand[j], f"demand_{j}"
    

Kapacity dodavatelů (S):  
4x<sub>11</sub> + 6x<sub>12</sub> + 5x<sub>13</sub> <= 35  
3x<sub>21</sub> + 2x<sub>22</sub> + 4x<sub>23</sub> <= 45  
  
∑<sub>j</sub>x<sub>ij</sub><= s<sub>i</sub>

```python
#     D1    D2    D3   kapacita
# S1  [4]   [6]   [5]  [35]
# S2  {3}   {2}   {4}  {45}
```
  
Bilance ve skladech tj. (spojení fází):  
*to co vydám ze skladu (S) = to co získá záklazník (C)*  
4x<sub>11</sub> + 3x<sub>21</sub> = 3y<sub>11</sub> + 4y<sub>12</sub> + 2y<sub>13</sub> + 3y<sub>14</sub>  
6x<sub>12</sub> + 2x<sub>22</sub> = 5y<sub>21</sub> + 2y<sub>22</sub> + 4y<sub>23</sub> + 1y<sub>24</sub>  
5x<sub>13</sub> + 4x<sub>23</sub> = 4y<sub>31</sub> + 3y<sub>32</sub> + 3y<sub>33</sub> + 2y<sub>34</sub>  
  
∑<sub>i</sub>x<sub>ij</sub> = ∑<sub>k</sub>y<sub>jk</sub>
  
```python
#     D1    D2    D3   kapacita
# S1  [4]   {6}   (5)  35
# S2  [3]   {2}   (4)  45

#     C1    C2    C3    C4
# D1  [3]   [4]   [2]   [3]
# D2  {5}   {2}   {4}   {1}
# D3  (4)   (3)   (3)   (2)
#     20     25    15    20
#        odbery
```

Poptávka zákazníka (C) = odběry:  
3y<sub>11</sub> + 5y<sub>21</sub> + 4y<sub>31</sub> = 20  
4y<sub>12</sub> + 2y<sub>22</sub> + 3y<sub>32</sub> = 25  
2y<sub>13</sub> + 4y<sub>23</sub> + 3y<sub>33</sub> = 15  
3y<sub>14</sub> + 1y<sub>24</sub> + 2y<sub>34</sub> = 20  
  
∑<sub>j</sub>y<sub>jk</sub> = d<sub>k</sub>
  
```python
#     C1    C2    C3    C4
# D1  [3]   {4}   (2)   -3-
# D2  [5]   {2}   (4)   -1-
# D3  [4]   {3}   (3)   -2-
#     [20]  {25}  (15)  -20-
#            odbery
```

In [8]:
# řešení
model.solve()

1

In [9]:
# řešení primalu
print(f"Stav: {LpStatus[model.status]}")
print(f"Min. náklady: {value(model.objective):.2f}\n")

print("Tok S → D:")
for i in suppliers:
    for k in depots:
        if x[i][k].varValue > 1e-6:
            print(f"  {i} → {k}: {x[i][k].varValue:.1f}")

print("\nTok D → C:")
for k in depots:
    for j in customers:
        if y[k][j].varValue > 1e-6:
            print(f"  {k} → {j}: {y[k][j].varValue:.1f}")

Stav: Optimal
Min. náklady: 390.00

Tok S → D:
  S1 → D1: 35.0
  S2 → D2: 45.0

Tok D → C:
  D1 → C1: 20.0
  D1 → C3: 15.0
  D2 → C2: 25.0
  D2 → C4: 20.0


min **c**<sup>T</sup>x  
s.t. A**x**<=**b**  
**x**>=0

max **b**^<sup>T</sup>y  
s.t. A<sup>T</sup>**y**>=**c**  
**y**<= 0  
  
-> primalová proměnná (počet) = duální omezení  
-> primálové omezení (maximum/minimum) = duální proměnná

In [10]:
# řešení dualu
print("\nSlacky a stínové ceny")
for con in model.constraints.values():
    print(f"{con.name:12s} | Slack={con.slack:6.1f} | Dual={con.pi:6.2f}")


Slacky a stínové ceny
supply_S1    | Slack=  -0.0 | Dual=  0.00
supply_S2    | Slack=  -0.0 | Dual= -1.00
balance_D1   | Slack=  -0.0 | Dual=  4.00
balance_D2   | Slack=  -0.0 | Dual=  3.00
balance_D3   | Slack=  -0.0 | Dual=  3.00
demand_C1    | Slack=  -0.0 | Dual=  7.00
demand_C2    | Slack=  -0.0 | Dual=  5.00
demand_C3    | Slack=  -0.0 | Dual=  6.00
demand_C4    | Slack=  -0.0 | Dual=  4.00


4x<sub>11</sub> + 6x<sub>12</sub> + 5x<sub>13</sub> <= 35  
3x<sub>21</sub> + 2x<sub>22</sub> + 4x<sub>23</sub> <= 45  
  
4x<sub>11</sub> + 3x<sub>21</sub> = 3y<sub>11</sub> + 4y<sub>12</sub> + 2y<sub>13</sub> + 3y<sub>14</sub>  
6x<sub>12</sub> + 2x<sub>22</sub> = 5y<sub>21</sub> + 2y<sub>22</sub> + 4y<sub>23</sub> + 1y<sub>24</sub>  
5x<sub>13</sub> + 4x<sub>23</sub> = 4y<sub>31</sub> + 3y<sub>32</sub> + 3y<sub>33</sub> + 2y<sub>34</sub>  
  
3y<sub>11</sub> + 5y<sub>21</sub> + 4y<sub>31</sub> = 20  
4y<sub>12</sub> + 2y<sub>22</sub> + 3y<sub>32</sub> = 25  
2y<sub>13</sub> + 4y<sub>23</sub> + 3y<sub>33</sub> = 15  
3y<sub>14</sub> + 1y<sub>24</sub> + 2y<sub>34</sub> = 20  

| Primální omezení       | Duální proměnná | omezení | Typ     | Význam                       |
| ---------------------- | --------------- | ------- | ------- | ---------------------------- |
| Kapacita dodavatele S1 | $u_1$           |    35   | $\le 0$ | stínová cena kapacity S1     |
| Kapacita dodavatele S2 | $u_2$           |    45   | $\le 0$ | stínová cena kapacity S2     |
| Rovnováha ve skladu D1 | $v_1$           |    0    | volná   | rovnováha toku přes sklad D1 |
| Rovnováha ve skladu D2 | $v_2$           |    0    | volná   | rovnováha toku přes sklad D2 |
| Rovnováha ve skladu D3 | $v_3$           |    0    | volná   | rovnováha toku přes sklad D3 |
| Poptávka zákazníka C1  | $w_1$           |    20   | volná   | stínová cena požadavku C1    |
| Poptávka zákazníka C2  | $w_2$           |    25   | volná   | stínová cena požadavku C2    |
| Poptávka zákazníka C3  | $w_3$           |    15   | volná   | stínová cena požadavku C3    |
| Poptávka zákazníka C4  | $w_4$           |    20   | volná   | stínová cena požadavku C4    |
  
  
  
maximalizuj = 35 u<sub>1</sub> + 45 u<sub>2</sub> + 20 w<sub>1</sub> + 25 w<sub>2</sub> + 15 w<sub>3</sub> + 20 w<sub>4</sub>  




📌 Podmínky optimálnosti:
Primal feasibility – primal splňuje všechna omezení

Dual feasibility – duál splňuje všechna omezení

Komplementární slacks (KKT podmínky) - Buď je v primalu rezerva (slack), nebo v duálu je „cena“ (stínová hodnota), ale ne oboje zároveň.

Solver:

sleduje, kolik z toho je využito → to je slack:  
*např. použijeme jen 30 → slack = 5*

současně počítá duální proměnnou u1, která říká:  
*„Kdybych měl o 1 jednotku víc (→ 36 místo 35), o kolik se zlepší optimum?“*  

👉 Pokud slack > 0 → duální proměnná = 0 (nemá cenu navyšovat)
👉 Pokud slack = 0 → duální proměnná může být záporná (cena té „hranice“)

Slacks → kolik volnosti zůstalo v primalu

Duální proměnné → jak „drahá“ je každá rovnice / omezení

Optimalní?  
| Krok                  | Co se kontroluje                                             |
| --------------------- | ------------------------------------------------------------ |
| 1. Primal feasibility | Všechna omezení primalu splněna                              |
| 2. Dual feasibility   | Všechna omezení duálu splněna                                |
| 3. Duality gap = 0    | $c^\top x - b^\top y = 0$                                    |
| 4. Komplementarita    | Pro každé omezení: buď slack > 0 nebo dual > 0, ale ne oboje |
